In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist.h5')
print("Saving the model as mnist.h5")

(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 13s 27ms/step - loss: 0.3149 - accuracy: 0.9020 - val_loss: 0.0664 - val_accuracy: 0.9808
Epoch 2/10
469/469 [==============================] - 14s 29ms/step - loss: 0.0902 - accuracy: 0.9735 - val_loss: 0.0430 - val_accuracy: 0.9858
Epoch 3/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0633 - accuracy: 0.9814 - val_loss: 0.0348 - val_accuracy: 0.9887
Epoch 4/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0509 - accuracy: 0.9855 - val_loss: 0.0338 - val_accuracy: 0.9889
Epoch 5/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0418 - accuracy: 0.9880 - val_loss: 0.0315 - val_accuracy: 0.9896
Epoch 6/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0374 - accuracy: 0.9890 - val_loss: 0.0332 - val_accuracy: 0.9895
Epoch 7/10
469/469 [==================

In [4]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load the trained model
model = load_model('mnist.h5')

def predict_digit(img):
    """Preprocess the image and predict the digit using the trained model."""
    img = img.convert('L')  # Convert to grayscale
    img = img.resize((28, 28))  # Resize to match the model input size
    img = np.array(img)

    # Invert colors if necessary (white on black → black on white)
    img = np.invert(img)

    img = img / 255.0  # Normalize pixel values (0 to 1)
    img = img.reshape(1, 28, 28, 1)  # Reshape to match model input

    # Predict digit
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # UI Elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid layout
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        """Clear the canvas."""
        self.canvas.delete("all")

    def classify_handwriting(self):
        """Capture the drawn image, preprocess, and classify it."""
        HWND = self.canvas.winfo_id()  # Get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # Get canvas coordinates
        a, b, c, d = rect

        # Adjust cropping to remove unwanted borders
        rect = (a + 10, b + 10, c - 10, d - 10)

        im = ImageGrab.grab(rect)  # Capture the drawing from canvas

        # Debugging: Save the image to check its quality
        im.save("debug_image.png")

        digit, acc = predict_digit(im)  # Get prediction
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    def draw_lines(self, event):
        """Draw lines on the canvas."""
        self.x = event.x
        self.y = event.y
        r = 4  # Smaller stroke size for better digit clarity
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

# Run the app
app = App()
mainloop()

1/1 [==============================] - 0s 19ms/step
